# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../WeatherPy/weatherdata.csv"
weatherdata_df = pd.read_csv(csv_path)
weatherdata_df


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.8000,-68.3000,46.40,76,75,4.61,AR,1616159129
1,New Norfolk,-42.7826,147.0587,60.01,69,0,1.10,AU,1616159130
2,Tasiilaq,65.6145,-37.6368,28.40,86,20,9.22,GL,1616159130
3,Ushuaia,-54.8000,-68.3000,46.40,76,75,4.61,AR,1616159129
4,Saint-Philippe,-21.3585,55.7679,82.99,74,90,23.02,RE,1616159130
...,...,...,...,...,...,...,...,...,...
584,Cherskiy,68.7500,161.3000,-30.01,100,0,2.80,RU,1616159275
585,Mataura,-46.1927,168.8643,55.00,88,100,5.01,NZ,1616159156
586,Punta Arenas,-53.1500,-70.9167,51.80,76,75,8.05,CL,1616159151
587,Mataura,-46.1927,168.8643,55.00,88,100,5.01,NZ,1616159156


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Store latitude and longitude in locations
locations = weatherdata_df[["Latitude", "Longitude"]]
humidity = weatherdata_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
newweatherdata_df = weatherdata_df.loc[(weatherdata_df["Max Temp"]<80) & (weatherdata_df["Max Temp"]>70) & (weatherdata_df["Wind Speed"]<10) & (weatherdata_df["Cloudiness"]==0)]
newweatherdata_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
175,Hong Kong,22.2855,114.1577,75.00,84,0,9.78,HK,1616158935
381,São Gabriel da Cachoeira,-0.1303,-67.0892,75.20,100,0,2.30,BR,1616159229
514,Emboscada,-25.1500,-57.3500,75.20,73,0,3.44,PY,1616159259
546,Dingle,10.9995,122.6711,78.73,77,0,6.29,PH,1616159266
558,Dingle,10.9995,122.6711,78.73,77,0,6.29,PH,1616159266
564,São Filipe,14.8961,-24.4956,74.80,65,0,6.08,CV,1616159271


In [11]:
hotel_df = newweatherdata_df[["City","Country","Latitude","Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
175,Hong Kong,HK,22.2855,114.1577,
381,São Gabriel da Cachoeira,BR,-0.1303,-67.0892,
514,Emboscada,PY,-25.1500,-57.3500,
546,Dingle,PH,10.9995,122.6711,
558,Dingle,PH,10.9995,122.6711,
564,São Filipe,CV,14.8961,-24.4956,


In [5]:
# cities = newweatherdata_df["City"]
# cities

175                   Hong Kong
381    São Gabriel da Cachoeira
514                   Emboscada
546                      Dingle
558                      Dingle
564                  São Filipe
Name: City, dtype: object

In [6]:
# find_duplicates = newweatherdata_df.duplicated(subset=['City'])
# find_duplicates

# newweatherdata_dropped = newweatherdata_df.drop([0,4])
# newweatherdata_dropped

# Unfortunately I only realised at this point I forgot to remove duplicates...

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
# Set params
params = {
    "radius": 5000,
    "types":"lodging",
    "key": gkey
}

# Iterate
for index,row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Run request
    response = requests.get(base_url, params = params)

    # print(response.url)
    hotel = response.json()
    #pprint(hotel)

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
   
    except:
        print("Missing result... skipping.")

hotel_df

,City,Country,Latitude,Longitude,Hotel Name
175,Hong Kong,HK,22.2855,114.1577,Mini Hotel Central
381,São Gabriel da Cachoeira,BR,-0.1303,-67.0892,Alarne - Área De Lazer Do Alto Rio Negro
514,Emboscada,PY,-25.1500,-57.3500,Quinta milagros
546,Dingle,PH,10.9995,122.6711,SEGAYA PROPERTY
558,Dingle,PH,10.9995,122.6711,SEGAYA PROPERTY
564,São Filipe,CV,14.8961,-24.4956,Tortuga B&B


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [38]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))